# Better Image Representation - Learn from NeRF<sup>1</sup>  and SRN<sup>2</sup> 

Thoughts from discussion with Qian   
Minghao   
Mar 31, 2021

(SNR is the followup to DeepVoxels<sup>3</sup>)   
1: [NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis](https://arxiv.org/abs/2003.08934)   
2: [Scene representation networks: Continuous 3D-structure-aware neural scene representations](https://papers.nips.cc/paper/2019/hash/b5dc4e5d9b495d0196f61d45b26ef33e-Abstract.html)   
3: [DeepVoxels: Learning Persistent 3D Feature Embeddings](https://vsitzmann.github.io/deepvoxels/)

These two neural networks has similar input, output, and pipeline. The pipeline can be written as:
\begin{align*}
    \mathbf{D} &= \mathcal{F}(\{\mathbf{X}\}) \\
    \mathbf{Y} &= \mathcal{G}(\mathbf{D}, \mathbf{V})
\end{align*}
Where 
- $\{\mathbf{X}\}$: Input, 100/many 2D images   
- Representation $\mathbf{D}$: one data chunk (array or weights) representing this single/class 3D object   
- Showed output $\{\mathbf{Y}\}$: 2D images of the object from given view points $\{\mathbf{V}\}$   
- $\mathcal{F}$: can be regarded as fusion and compression
- $\mathcal{G}$: can be regarded as extraction and projection.

SRN and NeRF has $\mathbf{D}$ in different format, thus different $\mathcal{F}$ and $\mathcal{G}$. NeRF's $\mathbf{D}$ is some weights for the network structure $\mathcal{G}$, and its $\mathcal{F}$ is the training process. SRN's $\mathbf{D}$ is more complex, as it aims to work for a class of objects, where a latent vector $z$ chooses which object to render.

Analyzing "entropy".   
In this section I use "entropy" in an informal manner, as a vague concept of how much information it contain. The numbers are not precise, just for comparision, saying that there's more/less information.

The input $\{\mathbf{X}\}$ has much redundancy. A human can grab a decent understanding of a 3D object if we show him/her just 3 or 4 images from different view points. The process $\{\mathbf{X}\} \rightarrow \mathbf{D}$ is more or less an overposed problem. 

As a result, the representation $\mathbf{D}$ takes much smaller disk space than input $\{\mathbf{X}\}$, but of course, larger than a single frame $\mathbf{X}$ in the input. 

Why this works so good, is that images from different viewpoints are closely related, but not easily convertible.

Its not that easy to find a 2D 

Why $\{\mathbf{X}\}$ and $\{\mathbf{Y}\}$ are all images? 2D sensor, 2D display. I bet if we have commonly fetched 3D display, the shown thing won't be images.

Currrently, all visual algorithms give "to-eye" results as 2D RGB pixel maps.